In [25]:
from get_response import PoliGPT
import rag_metrics as rm

In [26]:
preguntas = [
    "¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?",
    "¿Cuáles son los requisitos para estudiar en la Universidad?",
    "¿Qué carreras se ofrecen en la Universidad?",
    "¿Hay becas disponibles para estudiantes?",
    "¿Qué tipo de apoyo académico se ofrece a los estudiantes?",
    "¿Cómo se puede acceder a la información sobre las becas?",
]

In [27]:
poligpt = PoliGPT(faiss_index_dir='../01_data/project_faiss')
respuestas = []
for pregunta in preguntas:  
    respuesta = poligpt.query_poligpt(pregunta, k_context=2)
    if respuesta['response'] != 'No dispongo de información suficiente en el contexto proporcionado.':
        respuestas.append(respuesta)

FAISS inicializado - Vectores: 79044 | Dimensión: 768


In [28]:
def format_contexts(respuestas):
    """
    Formatea los contextos para valorar las métricas de RAG.
    """
    c = []
    for respuesta in respuestas:
        contexts = respuesta['contexts']
        c_aux = []
        for context in contexts:
            c_aux.append(context['content'])
        c.append(c_aux)
    return c

contextos = format_contexts(respuestas)
respuestas_final = [dic['response'] for dic in respuestas]
preguntas_final = [dic['query'] for dic in respuestas]

In [29]:
def evaluate_dataset(preguntas, respuestas, contextos):
    """Aggregate the three metrics over an iterable of RAG records."""
    g_scores, f1s, qc_sims = [], [], []
    for answer, contexts, question in zip(respuestas, contextos,preguntas):
        g_scores.append(rm.grounding_score(answer, contexts))
        f1s.append(rm.context_overlap_f1(answer, contexts))
        qc_sims.append(rm.question_context_similarity(question, contexts))
    n = max(len(g_scores), 1)
    return {
        "Grounding": sum(g_scores) / n,
        "ContextOverlapF1": sum(f1s) / n,
        "QuestionContextSim": sum(qc_sims) / n,
    }

In [30]:
evaluate_dataset(preguntas_final, respuestas_final, contextos)

{'Grounding': 0.5104868526846762,
 'ContextOverlapF1': 0.2762784210576794,
 'QuestionContextSim': 0.04147109444696931}